### In-Game Purchase Data Analysis

#### Background 

In this homework, you are ask to deal with a task of analyzing an "in-game purchase" dataset. In this game, players are able to to purchase optional items that enhance their playing experience. Now your task is to generate a report that breaks down the game's purchasing data into meaningful insights.

-----

#### Observable Trends

* There are 1163 active players. The vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%). 

* The age group that spends the most money is the 20-24 with 1,114.06 dollars as total purchase value and an average purchase of 4.32. In contrast, the demographic group that has the highest average purchase is the 35-39 with 4.76 and a total purchase value of 147.67. 
-----


In [48]:
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

# money style converter
def to_money(f: float):
    s = '{:.2f}'.format(f)
    if '.' in s:
        i = s.find('.')
    else:
        i = len(s)
    cnt = 0


    while i>0:
        if cnt == 3:
            cnt = 0
            s = s[:i] + ',' + s[i:]
        else:
            cnt += 1

        i-=1

    return '$' + s

## Player Count

* Display the total number of players


In [49]:
pd.DataFrame([len(purchase_data['SN'].unique())], columns=['Total Players'])

Total Players
0            576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [50]:
data = []
data.append(len(purchase_data['Item ID'].unique()))
data.append(to_money(purchase_data['Price'].mean()))
data.append(len(purchase_data))
# data.append('$' + str(purchase_data['Price'].sum())[0] + ',' + str(purchase_data['Price'].sum())[1:])
data.append(to_money(purchase_data['Price'].sum()))
pd.DataFrame([data], columns=['Number of Unique Items', 'Aberage Price', 'Number of Purchases', 'Total Revenue'])

Number of Unique Items Aberage Price  Number of Purchases Total Revenue
0                     183         $3.05                  780     $2,379.77

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [51]:
tot_count = purchase_data.drop_duplicates(subset=['SN']).groupby('Gender').count().apply(lambda x: x[0], axis=1).rename('Total Count')
percent = (tot_count/tot_count.sum() * 100).rename('Percentage of Players')
df = pd.concat([percent, tot_count], axis=1).sort_values('Total Count', ascending = False)
df['Percentage of Players'] = df.apply(lambda x: '{:.2f}'.format(x[0]), axis=1)
df.index.name = ''
df

Percentage of Players  Total Count
                                                        
Male                                  84.03          484
Female                                14.06           81
Other / Non-Disclosed                  1.91           11


## Purchasing Analysis (Gender)

* Run basic calculations to obtain below values by gender
  * Purchase Count
  * Average Purchase Price  
  * Total Purchase Value
  * Average Purchase Total per Person by Gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [52]:
temp = purchase_data.groupby(['SN', 'Gender'])
tot_price = temp.sum()['Price'].groupby('Gender').sum().rename('Total Purchase Value')
player_count = temp.sum()['Price'].groupby('Gender').count()
trans_count = purchase_data.groupby('Gender').count()['Purchase ID'].rename('Purchase Count')

df = pd.concat([trans_count, (tot_price/trans_count).rename('Average Purchase Price'), tot_price, (tot_price/player_count).rename('Average Purchase Total per Person')], axis=1)
df['Average Purchase Price'] = df['Average Purchase Price'].apply(lambda x: '{:.5f}'.format(x))
df['Average Purchase Total per Person'] = df['Average Purchase Total per Person'].apply(lambda x: to_money(x))
df

Purchase Count Average Purchase Price  \
Gender                                                         
Female                            113                3.20301   
Male                              652                3.01785   
Other / Non-Disclosed              15                3.34600   

                       Total Purchase Value Average Purchase Total per Person  
Gender                                                                         
Female                               361.94                             $4.47  
Male                                1967.64                             $4.07  
Other / Non-Disclosed                 50.19                             $4.56

## Age Demographics

* Categorize players using the age groups. Hint: use pd.cut()
  * each group is for 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* round the percentage column to two decimal points
* Display Age Demographics Table

In [53]:
score_cut = pd.cut(purchase_data.drop_duplicates(subset=['SN'])['Age'], [0] + [i for i in range(9, 40, 5)] + [np.inf], labels=['<10']+['{}-{}'.format(i, i+4) for i in range(10,36, 5)]+['40+'])
scores = pd.value_counts(score_cut, sort=False)
df = pd.DataFrame()
df['Percentage of Players'] = (scores/scores.sum() * 100).apply(lambda x: '{:.2f}'.format(x))
df['Total Count'] = scores
df

Percentage of Players  Total Count
<10                    2.95           17
10-14                  3.82           22
15-19                 18.58          107
20-24                 44.79          258
25-29                 13.37           77
30-34                  9.03           52
35-39                  5.38           31
40+                    2.08           12

## Purchasing Analysis (Age)

* The below each broken into groups of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [54]:
age_cut = pd.cut(purchase_data['Age'], [0] + [i for i in range(9, 40, 5)] + [np.inf], labels=['<10']+['{}-{}'.format(i, i+4) for i in range(10,36, 5)]+['40+'])
ages = pd.value_counts(age_cut, sort=False)
df = pd.DataFrame()
df['Purchase Count'] = ages
df['Average Purchase Price'] = [purchase_data[age_cut==age]['Price'].sum() for age in df.index]/ages
df['Total Purchase Value'] = (df['Average Purchase Price'] * ages).apply(to_money)
df['Aberage Purchase Total per Person'] = ([purchase_data[age_cut==age]['Price'].sum() for age in df.index]/scores).apply(to_money)
df['Average Purchase Price'] = df['Average Purchase Price'].apply(to_money)
df

Purchase Count Average Purchase Price Total Purchase Value  \
<10                23                  $3.35               $77.13   
10-14              28                  $2.96               $82.78   
15-19             136                  $3.04              $412.89   
20-24             365                  $3.05            $1,114.06   
25-29             101                  $2.90              $293.00   
30-34              73                  $2.93              $214.00   
35-39              41                  $3.60              $147.67   
40+                13                  $2.94               $38.24   

      Aberage Purchase Total per Person  
<10                               $4.54  
10-14                             $3.76  
15-19                             $3.86  
20-24                             $4.32  
25-29                             $3.81  
30-34                             $4.12  
35-39                             $4.76  
40+                               $3.19

## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value (top-5 players with highest total purchase value), then list (in a table):
  * SN(screen name)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [55]:
SNs = purchase_data['SN'].unique()
data = []
for SN in SNs:
    temp_df = purchase_data[purchase_data['SN'] == SN]
    data.append([len(temp_df), temp_df['Price'].sum()/len(temp_df), temp_df['Price'].sum()])
df = pd.DataFrame(data, index=SNs, columns=['Purchase Count', 'Average Purchase Price', 'Total Purchase Value'])
df.index.name = 'SN'
df = df.nlargest(n = 5,columns=['Total Purchase Value'])
df['Average Purchase Price'] = df['Average Purchase Price'].apply(to_money)
df['Total Purchase Value'] = df['Total Purchase Value'].apply(to_money)
df

Purchase Count Average Purchase Price Total Purchase Value
SN                                                                     
Lisosia93                 5                  $3.79               $18.96
Idastidru52               4                  $3.86               $15.45
Chamjask73                3                  $4.61               $13.83
Iral74                    4                  $3.40               $13.62
Iskadarya95               3                  $4.37               $13.10

## Most Popular Items

* Top 5 most popular items by purchase count:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [56]:
ids = purchase_data['Item ID'].unique()
data = []
names = []
for id in ids:
    temp_df = purchase_data[purchase_data['Item ID'] == id]
    names.append(temp_df['Item Name'].unique()[0])
    data.append([len(temp_df), temp_df['Price'].unique()[0], temp_df['Price'].sum()])
df = pd.DataFrame(data, columns=['Purchase Count', 'Item Price', 'Total Purchase Value'], index=[ids,names])
df['Item Price'] = df['Item Price'].apply(to_money)
df['Total Purchase Value'] = df['Total Purchase Value'].apply(to_money)
df.index.set_names(pd.Index(('Item ID', 'Item Name')), inplace=True)
df.nlargest(n=5, columns='Purchase Count').sort_index(axis=0, ascending=False).sort_values('Purchase Count', ascending=False)

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
108     Extraction, Quickblade Of Trembling Hands                  9   
82      Nirvana                                                    9   
92      Final Critic                                               8   

                                                     Item Price  \
Item ID Item Name                                                 
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
145     Fiery Glass Crusader                              $4.58   
108     Extraction, Quickblade Of Trembling Hands         $3.53   
82      Nirvana                                           $4.90   
92      Final Critic                                      $4.88   

                                                     Total Purchase Value  
Item ID Item Name                                                          
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
145     Fiery Glass Crusader                                       $41.22  
108     Extraction, Quickblade Of Trembling Hands                  $31.77  
82      Nirvana                                                    $44.10  
92      Final Critic                                               $39.04

## Most Profitable Items

* Top 5 most profitable items by total purchase value:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Sort the above table by total purchase value in descending order
* Display a preview of the data frame

In [57]:
ids = purchase_data['Item ID'].unique()
data = []
names = []
for id in ids:
    temp_df = purchase_data[purchase_data['Item ID'] == id]
    names.append(temp_df['Item Name'].unique()[0])
    data.append([len(temp_df), temp_df['Price'].unique()[0], temp_df['Price'].sum()])
df = pd.DataFrame(data, columns=['Purchase Count', 'Item Price', 'Total Purchase Value'], index=[ids,names])
df = df.nlargest(n=5, columns='Total Purchase Value')
df['Item Price'] = df['Item Price'].apply(to_money)
df['Total Purchase Value'] = df['Total Purchase Value'].apply(to_money)
df

Purchase Count Item Price  \
178 Oathbreaker, Last Hope of the Breaking Storm              12      $4.23   
82  Nirvana                                                    9      $4.90   
145 Fiery Glass Crusader                                       9      $4.58   
92  Final Critic                                               8      $4.88   
103 Singed Scalpel                                             8      $4.35   

                                                 Total Purchase Value  
178 Oathbreaker, Last Hope of the Breaking Storm               $50.76  
82  Nirvana                                                    $44.10  
145 Fiery Glass Crusader                                       $41.22  
92  Final Critic                                               $39.04  
103 Singed Scalpel                                             $34.80